<a href="https://colab.research.google.com/github/aminaalavi/AI-Projects/blob/main/AG2_CaptainAgent/AG2_(Autogen)_CaptainAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AG2 AI Agent - CaptainAgent

This notebook utilizes the AG2 (Autogen) AI agent called CaptainAgent, to search, retrieve, and analyze a publicly available McKinsey report from the web. CaptainAgent automatically spins up and orchestrates a specialized team of agents, each optimized for specific tasks, ensuring a comprehensive and high-quality analysis of the report's insights.

It has been setup in Google Colab for everyone to follow easily.

## 1. Install and Import Libraries
The following libraries are required to set up and run the AI agent. Ensure that all dependencies are installed before proceeding.

In [ ]:
!pip install ag2[captainagent] autogen -q
import os
import openai
import json
import autogen
from google.colab import drive
drive.mount('/content/drive')
from google.colab import userdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.3 MB/s eta 0:00:0

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


Mounted at /content/drive


## Importing Autogen Agents

This section imports the necessary components for setting up AI agents using the **Autogen framework**.

These imports set the foundation for building an AI-powered multi-agent system where:



*   The UserProxyAgent serves as an interface between the user and the AI agents.
*   The CaptainAgent spins up and orchestrates various agents to optimize task execution.





In [ ]:
from autogen import UserProxyAgent
from autogen.agentchat.contrib.captainagent import CaptainAgent

## Configuring OpenAI API Keys and Models

This section sets up API access to OpenAI models, ensuring secure authentication and smooth interaction with OpenAI's services.

#### API Key Configuration
- The OpenAI API key must be stored in Google Colab Secrets Manager before running this notebook. Also make sure your notebook has access to the Google Secrets


In [ ]:
# Configuring OpenAI API Keys and Models
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
config_list = [
    {
        "model": "gpt-4o-mini",
        "api_key": os.environ["OPENAI_API_KEY"]
        },
    {
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"]
    }
]

# Save to file
with open("/content/OAI_CONFIG_LIST.json", "w") as f:
    json.dump(config_list, f)

## Loading OpenAI Configuration from JSON

This section loads the OpenAI API configuration from a JSON file and filters the models to use only `gpt-4o-mini`.

In [ ]:
# set config path
config_path = "/content/OAI_CONFIG_LIST.json"
config_list = autogen.config_list_from_json(
    config_path, filter_dict={"model": ["gpt-4o-mini"]} #you can use any model of your choice. Make sure it is in the config file
)
os.environ["OAI_CONFIG_LIST"] = "/content/OAI_CONFIG_LIST.json"

## Configuring and Building AI Agents

This section defines the **LLM configuration** and initializes the **CaptainAgent** and **UserProxyAgent** for managing AI-driven tasks.


*   temperature: 0 → Ensures deterministic responses, minimizing randomness in model outputs.
*   config_list: config_list → Uses the pre-defined list of LLM configurations to select the appropriate model.

### CaptainAgent Configuration

The `CaptainAgent` is responsible for **managing and delegating tasks** to specialized sub-agents.

#### Key Parameters:
- **`name="captain_agent"`** → Assigns an identifier to the agent.
- **`llm_config=llm_config`** → Passes the predefined LLM settings.
- **`code_execution_config={"use_docker": False, "work_dir": "groupchat"}`**  
  - **`use_docker=False`** → Disables Docker-based execution.
  - **`work_dir="groupchat"`** → Defines the working directory for the agent.
- **`agent_config_save_path=None`** → If needed, a path can be specified to save agent configurations for reuse.

### UserProxyAgent Configuration

The `UserProxyAgent` acts as an **interface between the user and the AI system**, allowing seamless communication while ensuring controlled interactions.

#### Key Parameter:
- **`human_input_mode="NEVER"`** → Ensures that the agent operates **autonomously**, without requiring manual input from the user.







In [ ]:
# llm config
llm_config = {"temperature": 0, "config_list": config_list}

## build agents
captain_agent = CaptainAgent(
    name="captain_agent",
    llm_config=llm_config,
    code_execution_config={"use_docker": False, "work_dir": "groupchat"},
    agent_config_save_path=None,  # If you'd like to save the created agents in nested chat for further use, specify the save directory here
)
captain_user_proxy = UserProxyAgent(name="captain_user_proxy", human_input_mode="NEVER")

### Initiating a Chat with the CaptainAgent

The following code snippet demonstrates how the `UserProxyAgent` (`captain_user_proxy`) initiates a conversation with the `CaptainAgent` to retrieve insights on a given topic.

### Explanation of Chat Initialization Parameters

The `UserProxyAgent` (`captain_user_proxy`) interacts with the `CaptainAgent` to process and retrieve information. Below are the key parameters used in the chat initiation process.

#### Key Parameters:
- **`captain_user_proxy.initiate_chat(...)`**  
  Starts a chat session between the **UserProxyAgent** and the **CaptainAgent**.

- **`captain_agent`**  
  The AI agent responsible for **retrieving and analyzing** the requested information.

- **`message`**  
  The query that the agent should process.

- **`max_turns=1`**  
  Limits the conversation to **one exchange**, ensuring a **concise and direct response**.



In [ ]:
result = captain_user_proxy.initiate_chat(
    captain_agent,
    message="Retrieve McKinsey’s most recent analysis on generative AI, detailing its projected economic impact, industry-specific applications, and strategic recommendations for businesses. Identify key insights on revenue potential, productivity gains, and automation capabilities across sectors such as finance, healthcare, retail, and manufacturing. Highlight McKinsey’s quantified forecasts, case studies, and best practices for enterprise adoption, including implementation challenges, regulatory considerations, and workforce transformation strategies. Additionally, explore how organizations are leveraging generative AI to drive innovation and maintain competitive advantage in an evolving market.",
    max_turns=1,
)

captain_user_proxy (to captain_agent):

Retrieve McKinsey’s most recent analysis on generative AI, detailing its projected economic impact, industry-specific applications, and strategic recommendations for businesses. Identify key insights on revenue potential, productivity gains, and automation capabilities across sectors such as finance, healthcare, retail, and manufacturing. Highlight McKinsey’s quantified forecasts, case studies, and best practices for enterprise adoption, including implementation challenges, regulatory considerations, and workforce transformation strategies. Additionally, explore how organizations are leveraging generative AI to drive innovation and maintain competitive advantage in an evolving market.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

*****************************************************************************